# CUSTOM ORM MODULE
*COMPLIANT WITH DB-API 2.0 PYTHON SPECIFICATION MODULE*

In [254]:
# import dependencies
import sqlite3
import pandas as pd
from typing import TypeVar,Generic,_GenericAlias

In [255]:
# create a generic class
T = TypeVar('T')

In [256]:
# create connection
conn = sqlite3.connect(":memory:")

In [257]:
# create cursor / session
cursor = conn.cursor()

In [258]:
# create table statement
from typing import Any

DATATYPES = {
    'int':'INTEGER',
    'str':'TEXT',
    'float':'REAL',
    'bytes':'BLOB'
}

class Field(Generic[T]): # generics

    def __init__(
        self,
        default=None,
        primary_key:bool = False,
        required:bool = False,
        unique:bool = False,
    ) -> None:
        self.primary_key = primary_key
        self.required = required
        self.unique = unique
        self.value = default
        self.name = ''
        self.type = None
        self.query = ''

    def __info__(self):
        return vars(self)

    def __sql__(self) -> str: # return constraints field
        sql = ""
        if self.primary_key:
            sql += "PRIMARY KEY "
        if self.required:
            sql += "NOT NULL "
        if self.unique:
            sql += "UNIQUE "
        return sql.strip()
    
    def __bool__(self):
        return "hello"
    
    def __eq__(self, __value: object):

        if isinstance(__value,str):
            return f"{self.name} = '{__value}'"
        else:
            return f"{self.name} = {__value}"
        
    def __ne__(self, __value: object) -> str:
        
        if isinstance(__value,str):
            return f"NOT {self.name} = '{__value}'"
        else:
            return f"NOT {self.name} = {__value}"
        
    def __lt__(self,__value:object) -> str:
        
        if isinstance(__value,str):
            return f"{self.name} < '{__value}'"
        else:
            return f"{self.name} < {__value}" 
        
    def __gt__(self,__value:object) -> str:
        
        if isinstance(__value,str):
            return f"{self.name} > '{__value}'"
        else:
            return f"{self.name} > {__value}" 
        
    def __le__(self,__value:object) -> str:
        
        if isinstance(__value,str):
            return f"{self.name} <= '{__value}'"
        else:
            return f"{self.name} <= {__value}" 
        
    def __ge__(self,__value:object) -> str:
        
        if isinstance(__value,str):
            return f"{self.name} >= '{__value}'"
        else:
            return f"{self.name} >= {__value}" 
    
    def __repr__(self) -> str:
        return str(self.value)
    
    def __str__(self) -> str:
        return str(self.value)

class ForeignKey:

    def __init__(self) -> None:
        pass

class BaseModel:

    def __init__(self,**kwargs) -> None:
        pass

    def __sql__(self) -> str:

        columns = []

        for key,val in self.__class__.__annotations__.items():
            getattr(self,key).type = val.__args__[0]
            getattr(self,key).name = key
            if isinstance(val,_GenericAlias):
                columns.append(f'{key} {DATATYPES[val.__args__[0].__name__]} {getattr(self.__class__(),key).__sql__()}')

        return f"CREATE TABLE {self.__class__.__name__.lower()} ( {', '.join(columns)} );"

    def columns(self):
        return list(self.__class__.__annotations__.keys())

    def name(self):
        return self.__class__.__name__.lower()
    
    def col_info(self) -> dict:
        return {key:getattr(self,key).__info__() for key in self.columns()}

    pass

class User(BaseModel):
    id:Field[int] = Field(primary_key=True)
    username:Field[str] = Field(required=True,unique=True)
    password:Field[str] = Field(required=True)
    first_name:Field[str] = Field(required=True)
    last_name:Field[str] = Field(required=True)
    email:Field[str] = Field(required=True)

sql = """CREATE TABLE user ( id INTEGER PRIMARY KEY, username TEXT NOT NULL UNIQUE, password TEXT NOT NULL, first_name TEXT NOT NULL, last_name TEXT NOT NULL, email TEXT NOT NULL );"""
assert User().__sql__() == sql
User().col_info()

{'id': {'primary_key': True,
  'required': False,
  'unique': False,
  'value': None,
  'name': 'id',
  'type': int,
  'query': ''},
 'username': {'primary_key': False,
  'required': True,
  'unique': True,
  'value': None,
  'name': 'username',
  'type': str,
  'query': ''},
 'password': {'primary_key': False,
  'required': True,
  'unique': False,
  'value': None,
  'name': 'password',
  'type': str,
  'query': ''},
 'first_name': {'primary_key': False,
  'required': True,
  'unique': False,
  'value': None,
  'name': 'first_name',
  'type': str,
  'query': ''},
 'last_name': {'primary_key': False,
  'required': True,
  'unique': False,
  'value': None,
  'name': 'last_name',
  'type': str,
  'query': ''},
 'email': {'primary_key': False,
  'required': True,
  'unique': False,
  'value': None,
  'name': 'email',
  'type': str,
  'query': ''}}

In [259]:
# execute table statement
assert isinstance(cursor.execute(User().__sql__()),sqlite3.Cursor)

In [260]:
# select table
class Query:
    
    def __init__(self,table:BaseModel) -> None:
        self.table:BaseModel = table()
        self.query = ''
        self.values = []
        pass

    def select(self,*columns:list[Field],distinct:bool=False) -> None:

        if distinct:
            select_syntax = "SELECT DISTINCT {columns} FROM {table}"
        else:
            select_syntax = "SELECT {columns} FROM {table}"
        #all
        if len(columns) == 0:
            self.query = select_syntax.format(
                columns=', '.join(self.table.columns()),table=self.table.name()
            )
            return self

        #columns
        self.query = select_syntax.format(
            columns=', '.join([x.name for x in columns]),table=self.table.name()
        )

        return self
    
    def where(self,*conditions):

        self.query += f" WHERE {' and '.join(conditions)}"

        return self

    def insert(self,*columns:list[str],values:list=None):

        self.values = values
        
        if not columns:
            self.query += f" INSERT INTO {self.table.name()} ( {', '.join(self.table.columns())} ) VALUES ( {', '.join(self.__fillqMark(len(values)))} )"
        else:
            self.query += f" INSERT INTO {[self.table.name()]} ( {', '.join(self.__getColumnNames(columns))} ) VALUES ( {', '.join(self.__fillqMark(len(values)))} )"
        
        return self
    
    @staticmethod
    def _and(left,right):
        return f"{left} and {right}"
    
    @staticmethod
    def _or(left,right):
        return f"{left} or {right}"
    
    @staticmethod
    def _not(condition):
        return f"NOT {condition}"
    
    def __fillqMark(self,length:int):
        return ['?' for _ in range(length)]
    
    def __getColumnNames(self,iterable:list[Field]):
        return [f'{x.name}' for x in iterable]

    def __str__(self) -> str:
        return str(self.query).strip()
    
    def __repr__(self) -> str:
        return str(self.query).strip()

In [265]:
#session class
class Session:

    def __init__(self,engine:sqlite3.Connection) -> None:
        self.cursor = engine.cursor()
        self.connection = engine
        pass

    def __enter__(self):
        return self
    
    def __exit__(self,exc_type, exc_value, exc_traceback):
        self.cursor.close()
        return self
    
    def execute(self,sql:str,params=None):
        if params:
            self.cursor = self.cursor.execute(str(sql),params)
        else:
            self.cursor = self.cursor.execute(str(sql))
        return self
    
    def fetchall(self):
        return self.cursor.fetchall()
    
    def commit(self):
        return self.connection.commit()

Query(User).select().where(User.id < 1)

SELECT id, username, password, first_name, last_name, email FROM user WHERE id < 1